# Estagio CD&IA - Desafio


### Importação

In [2]:
from dbfread import DBF
import pandas as pd

In [3]:
# Caminho para o dbf file 
file_path = r'c:\Users\alexa\Downloads\pacigeral_jun24.dbf'

# Ler dbf file com encoding latin1
table = DBF(file_path, encoding='latin1')

In [4]:
import csv

In [5]:
# Arquivo novo, salvo em csv
output_csv = r'C:\Users\alexa\Downloads\pacigeral_jun24.csv'

# Abrir o arquivo CSV para escrita
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    
    # Escrever cabeçalhos das colunas (os nomes dos campos do DBF)
    writer.writerow(table.field_names)
    
    # Escrever os registros no arquivo CSV
    for record in table:
        writer.writerow(list(record.values()))

In [18]:
# Utilizar o arquivo csv criado
csv_file_path = r'C:\Users\alexa\Downloads\pacigeral_jun24.csv'

# Carregar o arquivo CSV no Pandas
df = pd.read_csv(csv_file_path)


# Teste! 

# Exibir as primeiras linhas do DataFrame
print(df.head())

# Exibir o formato do DataFrame (número de linhas e colunas)
print(f"Formato do DataFrame: {df.shape}")

# Exibir as colunas do DataFrame
print(f"Colunas: {df.columns}")

# Exibir informações sobre os tipos de dados e valores faltantes
print(df.info())



C:\Users\alexa\AppData\Local\Temp\ipykernel_4756\572354950.py:5: DtypeWarning: Columns (37,38,39,40,77,78,79,82,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


   INSTITU                   DSCINST  ESCOLARI  IDADE  SEXO UFNASC UFRESID  \
0    22950  HOSP CLINICAS DE MARILIA         3     78     2     SP      SP   
1   612374                     ICESP         3     49     1     BA      BA   
2     9326                   UNICAMP         9     50     1     SP      SP   
3    20737              FUND PIO XII         4     33     1     SP      SP   
4     9385   SANTA CASA DE SAO PAULO         9     23     1     SP      SP   

      IBGE                CIDADE  CATEATEND  ... REC04  IBGEATEN  \
0  3534500        OSCAR BRESSANE          2  ...   NaN   3529005   
1  2930709          SIMOES FILHO          2  ...   NaN   3550308   
2  3531209   MONTE ALEGRE DO SUL          2  ...   NaN   3509502   
3  3549409  SAO JOAQUIM DA BARRA          9  ...   NaN   3505500   
4  3542602              REGISTRO          2  ...   NaN   3550308   

                     DRSINST RRASINST  CIDADEINST HABILIT  \
0           DRS 09 - Marilia  RRAS 10     MARILIA       7   


###  Seleção

#### 1. Selecionar pacientes com Topografia de pulmão  (TOPOGRUP = 34)

In [19]:
df_pulmao = df[df.TOPOGRUP.isin(['C34'])]
df_pulmao.TOPOGRUP.value_counts()
df_pulmao.shape

(56800, 105)

#### 2. Selecionar pacientes com estado de residência de São Paulo ( UFRESID = SP)

In [20]:
df_pulmao = df_pulmao[df_pulmao.UFRESID == 'SP']
df_pulmao.UFRESID.value_counts()

UFRESID
SP    52205
Name: count, dtype: int64

#### 3. Selecionar pacientes com Base do Diagnóstico com Confirmação Microscópica (BASEDIAG = 3)

In [21]:
df_pulmao = df_pulmao[df_pulmao.BASEDIAG == 3]
df_pulmao.BASEDIAG.value_counts()

BASEDIAG
3    50583
Name: count, dtype: int64

#### 4. Retirar categorias 0, X e Y da coluna ECGRUP

In [22]:
df_pulmao = df_pulmao[~df_pulmao.ECGRUP.isin(['0', 'X', 'Y'])]
df_pulmao.ECGRUP.value_counts()

ECGRUP
IV     25611
III    11972
I       4864
II      2892
Name: count, dtype: int64

#### 5. Retirar pacientes que fizeram Hormonioterapia e TMO - TMO == 1 HORMONIO = 1

In [23]:
df_pulmao = df_pulmao[~((df_pulmao['HORMONIO'] == 1) | (df_pulmao['TMO'] == 1))]
print(df_pulmao.shape)


(45128, 105)


#### 6. Selecionar pacientes com Ano de Diagnóstico até 2019 (ANODIAG ¡= 2019)

In [24]:
df_pulmao = df_pulmao[~(df_pulmao.ANODIAG > 2019)]
df_pulmao.ANODIAG.value_counts().sort_index()

ANODIAG
2000    1382
2001    1367
2002    1474
2003    1571
2004    1693
2005    1811
2006    1633
2007    1569
2008    1725
2009    1963
2010    1950
2011    2073
2012    2098
2013    2306
2014    2412
2015    2346
2016    2439
2017    2410
2018    2261
2019    2199
Name: count, dtype: int64

#### 7. Retirar pacientes com IDADE menor do que 20 anos

In [25]:
df_pulmao = df_pulmao[df_pulmao.IDADE >= 20]
df_pulmao.IDADE.value_counts()
df_pulmao.shape

(38670, 105)

### Ajustes

#### 8. Calcular a diferen¸ca em dias entre Diagnóstico e Consulta, Tratamento e Diagnóstico,
Tratamento e Consulta, a partir dessas 3 datas (DTCONSULT, DTDIAG e DTTRAT).
Após o cálculo, codificar as colunas da seguinte forma:

• CONSDIAG – 0 = até 30 dias; 1 = entre 31 e 60 dias; 2 = mais de 61 dias;
1

• DIAGTRAT – 0 = até 60 dias; 1 = entre 61 e 90 dias; 2 = mais de 91 dias; 3 = Não tratou (datas de tratamento vazias);

• TRATCONS – 0 = at´e 60 dias; 1 = entre 61 e 90 dias; 2 = mais de 91 dias; 3 = Não tratou (datas de tratamento vazias).

In [26]:
list_datas = ['DTCONSULT', 'DTDIAG', 'DTTRAT']

for col_data in list_datas:
    df_pulmao[col_data] = pd.to_datetime(df_pulmao[col_data])

df_pulmao[list_datas].dtypes

DTCONSULT    datetime64[ns]
DTDIAG       datetime64[ns]
DTTRAT       datetime64[ns]
dtype: object

In [27]:
df_pulmao['CONSDIAG'] = (df_pulmao.DTDIAG - df_pulmao.DTCONSULT).dt.days
df_pulmao['DIAGTRAT'] = (df_pulmao.DTTRAT - df_pulmao.DTDIAG).dt.days
df_pulmao['TRATCONS'] = (df_pulmao.DTTRAT - df_pulmao.DTCONSULT).dt.days

In [28]:
# testar quando Nan tem, para poder conferir se os valores de 3 estao corretos 
df_pulmao[['CONSDIAG','DIAGTRAT', 'TRATCONS']].isna().sum()

CONSDIAG       0
DIAGTRAT    6390
TRATCONS    6390
dtype: int64

In [29]:
df_pulmao['CONSDIAG_CAT'] = [0 if consdiag <= 30 else 1 if consdiag <= 60 else 2 for consdiag in df_pulmao.CONSDIAG]

df_pulmao['DIAGTRAT_CAT'] = [0 if diagtrat <= 60 else 1 if diagtrat <= 90 else 2 if diagtrat >= 91 else 3 for diagtrat in df_pulmao.DIAGTRAT]

df_pulmao['TRATCONS_CAT'] = [0 if tratcons <= 60 else 1 if tratcons <=90 else 2 if tratcons >= 91 else 3 for tratcons in df_pulmao.TRATCONS]

print(df_pulmao['CONSDIAG_CAT'].value_counts())
print(df_pulmao['DIAGTRAT_CAT'].value_counts())
print(df_pulmao['TRATCONS_CAT'].value_counts())


CONSDIAG_CAT
0    32637
1     3413
2     2620
Name: count, dtype: int64
DIAGTRAT_CAT
0    23552
3     6390
2     4718
1     4010
Name: count, dtype: int64
TRATCONS_CAT
0    22568
3     6390
2     5582
1     4130
Name: count, dtype: int64


#### 9. Extrair somente o número das colunas DRS e DRSINST ( não achei DRSINSTITU nos dados)

In [34]:
print(df_pulmao.DRS.unique())
print(df_pulmao.DRSINST.unique())

['DRS 10  Piracicaba' 'DRS 03  Araraquara' 'DRS 01  Grande Sao Paulo'
 'DRS 07  Campinas' 'DRS 13  Ribeirao Preto' 'DRS 17  Taubate'
 'DRS 15  Sao Jose do Rio Preto' 'DRS 16  Sorocaba' 'DRS 09  Marilia'
 'DRS 04  Baixada Santista' 'DRS 06  Bauru' 'DRS 02  Aracatuba'
 'DRS 11  Presidente Prudente' 'DRS 08  Franca' 'DRS 05  Barretos'
 'DRS 14  Sao Joao da Boa Vista' 'DRS 12  Registro']
['DRS 10 - Piracicaba' 'DRS 03 - Araraquara' 'DRS 01 - Grande Sao Paulo'
 'DRS 07 - Campinas' 'DRS 13 - Ribeirao Preto' 'DRS 17 - Taubate'
 'DRS 15 - Sao Jose do Rio Preto' 'DRS 16 - Sorocaba' 'DRS 09 - Marilia'
 'DRS 04 - Baixada Santista' 'DRS 06 - Bauru' 'DRS 02 - Aracatuba'
 'DRS 11 - Presidente Prudente' 'DRS 05 - Barretos' 'DRS 08 - Franca'
 'DRS 12 - Registro' 'DRS 14 - Sao Joao da Boa Vista']


In [35]:
drs_expand = df_pulmao.DRS.str.split(' ', expand=True)
df_pulmao['nDRS'] = drs_expand[1].astype(int)
df_pulmao['nDRS'].value_counts()

nDRS
1     18158
15     2715
7      2684
6      2563
13     1754
10     1473
17     1294
9      1212
3      1167
16     1045
5      1019
2       850
8       718
14      711
4       700
11      532
12       75
Name: count, dtype: int64

In [36]:
drs_expand = df_pulmao.DRS.str.split(' ', expand=True)
df_pulmao['nDRSINST'] = drs_expand[1].astype(int)
df_pulmao['nDRSINST'].value_counts()

nDRSINST
1     18158
15     2715
7      2684
6      2563
13     1754
10     1473
17     1294
9      1212
3      1167
16     1045
5      1019
2       850
8       718
14      711
4       700
11      532
12       75
Name: count, dtype: int64